## 『本次練習內容』
#### 學習如何搭建Inception Block

## 『本次練習目的』
  #### 了解Inceotion原理
  #### 了解如何導入Inception block到原本架構中

![Incpeiton](img/Inception架構.png)

In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K
from keras import layers
from keras.layers import BatchNormalization
from keras.layers import Activation

Using TensorFlow backend.


## 導入InceptionV2-有BatchNormalization的Convolution

In [2]:
# padding = 'same', strides = (1,1), let h,w of output keep same as h,w of input 
def Conv2d_bn(x,filters,kernel_size,padding='same',strides=(1, 1),normalizer=True,activation='relu',name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    # 'channels_first' : corresponds to inputs with shape (batch_size, channels, height, width)
    # 'channels_last'  : corresponds to inputs with shape (batch_size, height, width, channels)
    if K.image_data_format() == 'channels_first':  
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
            filters, kernel_size,
            strides=strides, padding=padding,
            use_bias=False, name=conv_name)(x)
    if normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = Activation(activation, name=act_name)(x)
    return x

## 參考上圖搭建 InceptionV1_block

In [3]:
def InceptionV1_block(x, specs, channel_axis, name):  # x : img_input (=input_shape)
    (br0, br1, br2, br3) = specs   # specs : ((64,), (96,128), (16,32), (32,))
    # Conv2d_bn(img_input, filters, kernel_size)
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+"_Branch_0") # br0[0]:filters ; kernel_size=(1, 1)
    
    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1") # br1[0]:filters ; kernel_size=(1, 1)
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 3), name=name+"_Branch_1_1") # br1[1]:filters ; kernel_size=(3, 3)
    
    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2") # br2[0]:filters ; kernel_size=(1, 1)
    branch_2 = Conv2d_bn(branch_2, br2[1], (5, 5), name=name+"_Branch_2_1") # br2[1]:filters ; kernel_size=(5, 5)
    
    # MaxPooling2D : MaxPooling2D()(x)
    branch_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same', name=name+"_Branch_3")(x)
    branch_3 = Conv2d_bn(branch_3, br3[0], (1, 1), name=name+"_Branch_3_1") # br3[1]:filters ; kernel_size=(1, 1)
    
    # concatenate, (64+128+32+32=256)
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis = channel_axis,
        name = name+"_Concatenated")
    return x


## 測試 InceptionV1_block

In [4]:
img_input = Input(shape=(224,224,1))
x=InceptionV1_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1') # name = 'Block_1' 
print(x)  # Block_1_Concatenated

Tensor("Block_1_Concatenated/concat:0", shape=(None, 224, 224, 256), dtype=float32)


## 將 InceptionV1_block中n*n卷積改為1 x n+n x 1

In [5]:
def InceptionV3_block(x, specs, channel_axis, name):  # x : img_input (=input_shape)
    (br0, br1, br2, br3) = specs   # specs : ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+"_Branch_0") # br0[0]:filters ; kernel_size=(1, 1)
    
    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1") # br1[0]:filters ; kernel_size=(1, 1)

    # (3, 3) kernel_size change to -> (1, 3) + (3, 1) :
    branch_1 = Conv2d_bn(branch_1, br1[1], (1, 3), name=name+"_Branch_1_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 1), name=name+"_Branch_1_2")
        
    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2") # br2[0]:filters ; kernel_size=(1, 1)

    # (5, 5) kernel_size change to -> (1, 5) + (5, 1) :
    branch_2 = Conv2d_bn(branch_2, br2[1], (1, 5), name=name+"_Branch_2_1")
    branch_2 = Conv2d_bn(branch_2, br2[1], (5, 1), name=name+"_Branch_2_2")
    
    # MaxPooling2D : MaxPooling2D()(x)
    branch_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same', name=name+"_Branch_3")(x)
    branch_3 = Conv2d_bn(branch_3, br3[0], (1, 1), name=name+"_Branch_3_1") # br3[1]:filters ; kernel_size=(1, 1)
    
    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis = channel_axis,
        name = name+"_Concatenated")
    return x


## 測試

In [6]:
img_input = Input(shape=(224,224,1))
x=InceptionV3_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1') # name = 'Block_1' 
print(x)  # Block_1_Concatenated

Tensor("Block_1_Concatenated_1/concat:0", shape=(None, 224, 224, 256), dtype=float32)


## 額外練習

## 將VGG16 Block_3中的Convolution全部改為InceptionV1_block
## Block_5中的Convolution全部改為InceptionV3_block
## 並將所有Convolution改為Conv2d_bn

In [7]:
# modification from vgg16 : 
import numpy as np
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K

# image input shape=(244,244,1) means (h,w,c)
def VGG16_Inception(include_top=True, input_tensor=None, input_shape=(224,224,1),
          pooling='max', classes=1000) :
 
    # use functional input :
    img_input = Input(shape=input_shape) 
    
    # change all Convolution to Conv2d_bn,
    
    # Block 1
    # Conv2d_bn(img_input, filters, kernel_size)
    x = Conv2d_bn(img_input, 64, (3, 3), activation='relu', padding='same', name='block1_conv1') 
    x = Conv2d_bn(x, 64, (3, 3), activation='relu', padding='same', name='block1_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x) # (pool_size=(2,2)

    # Block 2
    x = Conv2d_bn(x, 128, (3, 3), activation='relu', padding='same', name='block2_conv1')
    x = Conv2d_bn(x, 128, (3, 3), activation='relu', padding='same', name='block2_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x) # (pool_size=(2,2)

    # Block 3
    # change Convolution of Block-3  to InceptionV1_block
    x=InceptionV1_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'InceptionV1_block3_1')
    x=InceptionV1_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'InceptionV1_block3_2')
    x=InceptionV1_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'InceptionV1_block3_3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x) # (pool_size=(2,2)
    
    # Block 4
    x = Conv2d_bn(x, 512, (3, 3), activation='relu', padding='same', name='block4_conv1')
    x = Conv2d_bn(x, 512, (3, 3), activation='relu', padding='same', name='block4_conv2')
    x = Conv2d_bn(x, 512, (3, 3), activation='relu', padding='same', name='block4_conv3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x) # (pool_size=(2,2)
      
    # Block 5
    # change Convolution of Block-5  to InceptionV3_block
    x=InceptionV3_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'InceptionV3_block5_1')
    x=InceptionV3_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'InceptionV3_block5_2')
    x=InceptionV3_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'InceptionV3_block5_3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x) # (pool_size=(2,2)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x) # flatten
        # FC layer        
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        # output layer
        x = Dense(classes, activation='softmax', name='predictions')(x) # classes=1000, class >2 so use 'softmax'
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x) # flatten
        elif pooling == 'max':              
            x = GlobalMaxPooling2D()(x)     # flatten

    inputs = img_input
    # Create model, x means the outputs which after process of convolution layer
    model = Model(inputs, x, name='VGG16__Inception')
   
    return model

In [8]:
model = VGG16_Inception(include_top=False)

In [9]:
model.summary()

Model: "VGG16__Inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
block1_conv1_conv (Conv2D)      (None, 224, 224, 64) 576         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 224, 224, 64) 192         block1_conv1_conv[0][0]          
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 224, 224, 64) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________